In [544]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from  more_itertools import unique_everseen
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import metrics

Преподаватель: Алексей Кузьмин

    Использовать dataset MovieLens
    Построить рекомендации (регрессия, предсказываем оценку) на фичах:

    TF-IDF на тегах и жанрах
    Средние оценки (+ median, variance, etc.) пользователя и фильма

    Оценить RMSE на тестовой выборке



In [425]:
movies = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/movies.csv')
ratings = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/ratings.csv')
tags = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/tags.csv')

In [426]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [427]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [428]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


В таблице с тегами надо создать колонку с тегами, записанными в строку

In [429]:
tags_series=tags.groupby([tags['userId'],tags['movieId']])['tag'].apply(list)

In [430]:
new_tags=[]
for i in range(len(tags)):
    user=tags['userId'].iloc[i]
    movie=tags['movieId'].iloc[i]
    ts=tags_series[user,movie]
    new_tags.append(ts)

In [431]:
tags['new_tags']=new_tags

In [432]:
tags['new_tags']=tags['new_tags'].apply(lambda row: str(row))

In [433]:
del tags['tag']
del tags['timestamp']

In [434]:
tags=tags.drop_duplicates()

In [435]:
tags.head()

,userId,movieId,new_tags
0,2,60756,"['funny', 'Highly quotable', 'will ferrell']"
3,2,89774,"['Boxing story', 'MMA', 'Tom Hardy']"
6,2,106782,"['drugs', 'Leonardo DiCaprio', 'Martin Scorsese']"
9,7,48516,['way too long']
10,18,431,"['Al Pacino', 'gangster', 'mafia']"


Таблиы tags, movies, ratings соединим по двум ключам: userId, movieId

In [436]:
tags_raitings=pd.merge(tags, ratings, on=['userId','movieId'])

In [437]:
movies_tags_raitings=pd.merge(tags_raitings, movies, on='movieId')

In [438]:
del movies_tags_raitings['timestamp']

In [439]:
def good_str(a):
    b=a.replace('[','')
    c=b.replace(']','')
    d=c.replace(', ',' ')
    e=d.replace('|',' ')
    return e

In [440]:
movies_tags_raitings['tags']=movies_tags_raitings['new_tags'].apply(lambda row: good_str(row))

In [441]:
del movies_tags_raitings['new_tags']

In [442]:
movies_tags_raitings['genres']=movies_tags_raitings['genres'].apply(lambda row: good_str(row))

In [443]:
movies_tags_raitings.head()

,userId,movieId,rating,title,genres,tags
0,2,60756,5.0,Step Brothers (2008),Comedy,'funny' 'Highly quotable' 'will ferrell'
1,62,60756,3.5,Step Brothers (2008),Comedy,'comedy' 'funny' 'will ferrell'
2,424,60756,4.0,Step Brothers (2008),Comedy,'funny' 'will ferrell'
3,2,89774,5.0,Warrior (2011),Drama,'Boxing story' 'MMA' 'Tom Hardy'
4,2,106782,5.0,"Wolf of Wall Street, The (2013)",Comedy Crime Drama,'drugs' 'Leonardo DiCaprio' 'Martin Scorsese'


Применим tfidf к тегам и жанрам

In [444]:
tfidf = TfidfVectorizer()

In [445]:
genres_tfidf=tfidf.fit_transform(movies_tags_raitings['genres'])

In [446]:
tfidf.vocabulary_

{'comedy': 4,
 'drama': 7,
 'crime': 5,
 'thriller': 21,
 'war': 22,
 'mystery': 16,
 'documentary': 6,
 'action': 0,
 'sci': 20,
 'fi': 9,
 'imax': 13,
 'adventure': 1,
 'children': 3,
 'fantasy': 8,
 'animation': 2,
 'horror': 12,
 'romance': 19,
 'musical': 15,
 'western': 23,
 'film': 10,
 'noir': 18,
 'no': 17,
 'genres': 11,
 'listed': 14}

In [447]:
genres_tfidf=genres_tfidf.toarray()

In [448]:
genres_tfidf.shape

(1635, 24)

In [449]:
tags_tfidf=tfidf.fit_transform(movies_tags_raitings['tags'])

In [450]:
tfidf.vocabulary_

{'funny': 597,
 'highly': 705,
 'quotable': 1222,
 'will': 1677,
 'ferrell': 549,
 'comedy': 321,
 'boxing': 198,
 'story': 1447,
 'mma': 999,
 'tom': 1553,
 'hardy': 674,
 'drugs': 465,
 'leonardo': 890,
 'dicaprio': 419,
 'martin': 949,
 'scorsese': 1332,
 'stock': 1441,
 'market': 946,
 'wall': 1641,
 'street': 1454,
 'way': 1653,
 'too': 1554,
 'long': 909,
 'suspense': 1483,
 'twist': 1586,
 'ending': 495,
 'undercover': 1593,
 'cop': 345,
 'atmospheric': 112,
 'jack': 813,
 'nicholson': 1062,
 'al': 46,
 'pacino': 1120,
 'gangster': 607,
 'mafia': 932,
 'holocaust': 722,
 'true': 1578,
 'anthony': 84,
 'hopkins': 729,
 'courtroom': 355,
 'drama': 458,
 'britpop': 206,
 'indie': 776,
 'record': 1243,
 'label': 874,
 'music': 1036,
 'dumpster': 469,
 'diving': 437,
 'sustainability': 1485,
 'bloody': 179,
 'bromance': 211,
 'james': 819,
 'franco': 583,
 'seth': 1351,
 'rogen': 1281,
 'black': 172,
 'hole': 718,
 'sci': 1327,
 'fi': 553,
 'time': 1543,
 'travel': 1569,
 'bad': 131,

In [451]:
tags_tfidf=tags_tfidf.toarray()

In [452]:
tags_tfidf.shape

(1635, 1708)

In [458]:
tags_tfidf = pd.DataFrame(data=tags_tfidf)
genres_tfidf = pd.DataFrame(data=genres_tfidf)

In [460]:
tfidf=pd.concat([tags_tfidf,genres_tfidf],axis=1)

In [487]:
data=pd.concat([tfidf,movies_tags_raitings],axis=1)

Посчитаем медианные оценки для каждого пользователя и фильма

In [489]:
median_rating_user=data['rating'].groupby(data['userId']).median()

In [490]:
median_rating_movie=data['rating'].groupby(data['movieId']).median()

In [491]:
data['median_rating_user']=data['userId'].apply(lambda row: median_rating_user[row])

In [492]:
data['median_rating_movie']=data['movieId'].apply(lambda row: median_rating_movie[row])

Разделим данные на параметры и таргет

In [493]:
y=data['rating']

In [494]:
X=data

In [501]:
del X['userId']
del X['movieId']
del X['title']
del X['tags']
del X['rating']
del X['genres']

Рейтиги похожи на классы. После tfidf очень много колонок с пустотами, удобнее использовать дерево решений.

In [532]:
le = preprocessing.LabelEncoder()
le.fit(y)
y=le.transform(y)

In [542]:
print(y.max(),', ',y.min())

9 ,  0


Надо предсказать метку класса. Всего классов 9

In [507]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [523]:
tree = DecisionTreeClassifier()

In [535]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [537]:
y_pred=tree.predict(X_test)

In [545]:
metrics.accuracy_score(y_pred,y_test)

0.8287461773700305